In [1]:
from __future__ import print_function
import tensorflow as tf
import os

# Dataset Parameters
MODE = 'folder' # or 'file', if you chose a plain text file
DATASET_PATH = 'path/to/dataset'

# Image parameters
N_CLASSES = 2
IMG_HIGHT = 64
IMG_WIDTH = 64
CHANNELS = 3

In [ ]:
# Reading the dataset
# 2 modes: 'file' or 'folder'
def read_images(dataset_path, mode, batch_size):
    imagepaths, labels = list(), list()
    if mode == 'file':
        # read the dataset file
        with open(dataset_path) as f:
            data = f.read.splitlines()
        for d in data:
            imagepaths.append(d.split(' ')[0])
            labels.append(int(d.split(' ')[1]))
    elif mode == 'folder':
        # an id will be affected to each sub-folders by alphabetical order
        label = 0
        # list the directory
        try: # python2
            classes = sorted(os.walk(dataset_path).next()[1])
        except Exception:
            classes = sorted(os.walk(dataset_path).__next__()[1])
        # list each sub-directory
        for c in classes:
            c_dir = os.path.join(dataset_path, c)
            try: # python2
                walk = os.walk(c_dir).next()
            except Exception:
                walk = os.walk(c_dir).__next__()
            # add each images to the training set
            for sample in walk[2]:
                # only keep jpeg images
                if sample.endswith('.jpg') or sample.endswith('.jpeg'):
                    imagepaths.append(os.path.join(c_dir, sample))
                    labels.append(label)
                label += 1
    else:
        raise Exception("Unknown mode.")
        
    # convert to tensor
    imagepaths = tf.convert_to_tensor(imagepaths, dtype=tf.string)
    labels = tf.convert_to_tensor(labels, dtype=tf.int32)
    
    # build a tf queue, shuffle data
    image, label = tf.train.slice_input_producer([imagepaths, label], shuffle=True)
    
    # read image from disk
    image = tf.read_file(image)
    image = tf.imag.decode_jpeg(image, channels=CHANNELS)
    
    # resize image to a common size
    image = tf.imag.resize_images(image,[IMG_HEIGHT, IMG_WIDTH])
    
    # normalisation
    image = image * 1.0 / 127.5 - 1.0
    
    # create batch X, y
    X, y = tf.train.batch([image, label], batch_size=batch_size,
                          capacity=batch_size*8,
                          num_threads=5)
    return X, y